In [1]:
# All 'mp' and 'hands' related lines have been removed.
# Other necessary imports like cv2, os, tensorflow, numpy are assumed to be present.
import cv2
import os
import tensorflow as tf
import numpy as np

def run_detection():
    # --- Load the saved model and class names ---
    model_path = 'asl_model.h5'
    class_names_path = 'class_names.txt'
    IMG_SIZE = (64, 64) # Ensure this matches the training configuration

    if not os.path.exists(model_path):
        print(f"Error: Model file not found at '{model_path}'")
        return
    else:
        model = tf.keras.models.load_model(model_path)
        print(f"Model '{model_path}' loaded successfully.")

        if not os.path.exists(class_names_path):
            print(f"Error: Class names file not found at '{class_names_path}'")
            return
        else:
            with open(class_names_path, 'r') as f:
                class_names = [line.strip() for line in f.readlines()]
            print(f"Class names loaded successfully: {class_names}")

    # --- Real-time Webcam Hand Sign Detection ---

    print("\n--- Starting Webcam for Real-time Detection ---")
    print("Place your hand inside the green box.")
    print("Press 'q' to quit the webcam window.")

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        exit()

    # Define the Region of Interest (ROI) box
    box_size = 300
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't read initial frame.")
        exit()
    height, width, _ = frame.shape

    x1 = int((width - box_size) / 2)
    y1 = int((height - box_size) / 2)
    x2 = x1 + box_size
    y2 = y1 + box_size

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting ...")
            break

        frame = cv2.flip(frame, 1)

        # Extract the ROI for prediction
        roi = frame[y1:y2, x1:x2]

        # --- Preprocess the ROI for the model ---
        rgb_roi_model = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        resized_roi = cv2.resize(rgb_roi_model, IMG_SIZE)
        img_array = tf.keras.utils.img_to_array(resized_roi)
        img_array = tf.expand_dims(img_array, 0)

        # --- Make a prediction on the ROI ---
        predictions = model.predict(img_array, verbose=0)
        predicted_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_index]
        confidence = 100 * np.max(predictions[0])

        # --- Display the prediction and the ROI box on the main frame ---
        display_text = f"Prediction: {predicted_class} ({confidence:.1f}%)"
        cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('ASL Hand Sign Detection (Press Q to Quit)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("--- Webcam feed closed ---")

# To run the real-time detection
run_detection()

Model 'asl_model.h5' loaded successfully.
Class names loaded successfully: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

--- Starting Webcam for Real-time Detection ---
Place your hand inside the green box.
Press 'q' to quit the webcam window.
--- Webcam feed closed ---
